In [54]:
import os
import warnings
import keras
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import newaxis
from keras.layers.core import  Activation, Dropout
from keras.models import Model
from keras.layers import Input, Embedding,LSTM, Dense,TimeDistributed
from keras import metrics
from sklearn.metrics import mean_squared_error
from math import sqrt
from numpy import concatenate
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [55]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(),list()
    for i in range(n_in, 0 ,-1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1,i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = concat(cols, axis =1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace = True)
    return agg

def plot_results(predicted_data, true_data):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    plt.plot(predicted_data, label='Prediction')
    plt.legend()
    plt.show()
def predict_sequences_full(model, data, window_size):
    #Shift the window by 1 new prediction each time, re-run predictions on new window
    curr_frame = data[0]
    predicted = []
    for i in range(len(data)):
        predicted.append(model.predict(curr_frame[newaxis,:,:])[0,0])
        curr_frame = curr_frame[1:]
        curr_frame = np.insert(curr_frame, window_size-1,predicted[-1], axis=0)
    return predicted

In [56]:
dataset_stock = read_csv('fai1.csv', header = 0, index_col=0)
values_stock = dataset_stock.values
encoder = LabelEncoder()
values_stock[:,0] = encoder.fit_transform(values_stock[:,0])
values_stock = values_stock.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_stock = scaler.fit_transform(values_stock)
reframed_stock = series_to_supervised(scaled_stock, 1, 1)
print(reframed_stock.head())

   var1(t-1)   var1(t)
1   0.182609  0.191304
2   0.191304  0.182609
3   0.182609  0.191304
4   0.191304  0.182609
5   0.182609  0.104348


In [57]:
dataset_gap = read_csv('gap1.csv', header = 0, index_col=0)
values_gap = dataset_gap.values
encoder = LabelEncoder()
values_gap[:,0] = encoder.fit_transform(values_gap[:,0])
values_gap = values_gap.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_gap = scaler.fit_transform(values_gap)
reframed_gap = series_to_supervised(scaled_gap, 1, 1)
print(reframed_gap.head())


   var1(t-1)   var1(t)
1   0.488038  0.435407
2   0.435407  0.617225
3   0.617225  0.401914
4   0.401914  0.559809
5   0.559809  0.397129


In [58]:
dataset_fai = read_csv('fai1.csv', header = 0, index_col=0)
values_fai = dataset_fai.values
encoder = LabelEncoder()
values_fai[:,0] = encoder.fit_transform(values_fai[:,0])
values_fai = values_fai.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_fai = scaler.fit_transform(values_fai)
reframed_fai = series_to_supervised(scaled_fai, 1, 1)
print(reframed_fai.head())

   var1(t-1)   var1(t)
1   0.182609  0.191304
2   0.191304  0.182609
3   0.182609  0.191304
4   0.191304  0.182609
5   0.182609  0.104348


In [59]:
values_stock = reframed_stock.values
n_train_hours = 80
n_end = n_train_hours + 5
train_stock = values_stock[:n_train_hours, :]

test_stock = values_stock[n_train_hours:n_end :]

train_X_stock, train_y_stock = train_stock[:,:-1], train_stock[:, -1]

test_stock_X, test_stock_y = test_stock[:,:-1],test_stock[:,-1]

train_X_stock = train_X_stock.reshape((train_X_stock.shape[0], 1, train_X_stock.shape[1]))
test_stock_X = test_stock_X.reshape((test_stock_X.shape[0], 1, test_stock_X.shape[1]))
print(train_X_stock.shape, train_y_stock.shape, test_stock_X.shape, test_stock_y.shape)
print(test_stock_X, test_stock_y)

(80, 1, 1) (80,) (5, 1, 1) (5,)
[[[ 0.44347823]]

 [[ 0.44347823]]

 [[ 0.44347823]]

 [[ 0.44347823]]

 [[ 0.34782606]]] [ 0.44347823  0.44347823  0.44347823  0.34782606  0.34782606]


In [60]:
values_gap = reframed_gap.values

train_gap = values_gap[:n_train_hours, :]

test_gap = values_gap[n_train_hours:n_end :]

train_X_gap, train_y_gap = train_gap[:,:-1], train_gap[:, -1]

test_gap_X, test_gap_y = test_gap[:,:-1],test_gap[:,-1]

train_X_gap = train_X_gap.reshape((train_X_gap.shape[0], 1, train_X_gap.shape[1]))
test_gap_X = test_gap_X.reshape((test_gap_X.shape[0], 1, test_gap_X.shape[1]))
print(train_X_gap.shape, train_y_gap.shape, test_gap_X.shape, test_gap_y.shape)
print(test_gap_X, test_gap_y)

(80, 1, 1) (80,) (5, 1, 1) (5,)
[[[ 0.5119617 ]]

 [[ 0.507177  ]]

 [[ 0.52153111]]

 [[ 0.56937796]]

 [[ 0.45933014]]] [ 0.507177    0.52153111  0.56937796  0.45933014  0.39234447]


In [61]:
values_fai = reframed_fai.values

train_fai = values_fai[:n_train_hours, :]

test_fai = values_fai[n_train_hours:n_end :]

train_X_fai, train_y_fai = train_fai[:,:-1], train_fai[:, -1]

test_fai_X, test_fai_y = test_fai[:,:-1],test_fai[:,-1]

train_X_fai = train_X_fai.reshape((train_X_fai.shape[0], 1, train_X_fai.shape[1]))
test_fai_X = test_fai_X.reshape((test_fai_X.shape[0], 1, test_fai_X.shape[1]))
print(train_X_fai.shape, train_y_fai.shape, test_fai_X.shape, test_fai_y.shape)
print(test_fai_X, test_fai_y)

(80, 1, 1) (80,) (5, 1, 1) (5,)
[[[ 0.44347823]]

 [[ 0.44347823]]

 [[ 0.44347823]]

 [[ 0.44347823]]

 [[ 0.34782606]]] [ 0.44347823  0.44347823  0.44347823  0.34782606  0.34782606]


In [62]:
stock_input = Input(shape=(train_X_stock.shape[1], train_X_stock.shape[2]), dtype='float32', name='stock_input')
print (stock_input)

Tensor("stock_input_2:0", shape=(?, 1, 1), dtype=float32)


In [63]:
gap_input = Input(shape=(train_X_gap.shape[1], train_X_gap.shape[2]), dtype='float32', name='gap_input')
print (gap_input)

Tensor("gap_input_2:0", shape=(?, 1, 1), dtype=float32)


In [64]:
fai_input = Input(shape=(train_X_fai.shape[1], train_X_fai.shape[2]), dtype='float32', name='fai_input')
print (fai_input)

Tensor("fai_input_2:0", shape=(?, 1, 1), dtype=float32)


In [65]:
merg_first = keras.layers.concatenate([stock_input, gap_input, fai_input])
#merg_first = Embedding(output_dim=512, input_dim = 10000, input_length=1)(merg_first)
print (merg_first)

Tensor("concatenate_5/concat:0", shape=(?, 1, 3), dtype=float32)


In [66]:
lstm_out = LSTM(5)(merg_first)
print(lstm_out)
lstm_out = Embedding(output_dim=512, input_dim = 10000, input_length=1)(lstm_out)
lstm_out = LSTM(100)(lstm_out)
lstm_out = Dense(1)(lstm_out)
print (lstm_out)  

Tensor("lstm_13/TensorArrayReadV3:0", shape=(?, 5), dtype=float32)
Tensor("dense_5/BiasAdd:0", shape=(?, 1), dtype=float32)


In [67]:
output = Dense(1, activation='linear', name='output')(lstm_out)

In [68]:
model = Model(inputs=[stock_input, gap_input, fai_input], outputs= output)

In [69]:
model.compile(loss="mse", optimizer="rmsprop")

In [70]:
validation_data=({'stock_input':test_stock_X, 
               'gap_input': test_gap_X, 
               'fai_input': test_fai_X},
              {'output':test_fai_y})
history=model.fit({'stock_input':train_X_stock, 
           'gap_input': train_X_gap, 
           'fai_input': train_X_fai},
         {'output':train_y_fai},
         epochs=30, batch_size=1, validation_data=validation_data, verbose=2, shuffle=False)





ValueError: Tried to convert 'x' to a tensor and failed. Error: None values not supported.